Notes:
- The state CT is not listed as counties in "CBSA.xlsx", but instead as "planning regions". As a result, CT is not in my sample. As a final step, clean up on that and augment the "CBSA" file with CT. Check if any other states has this problem.


In [237]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import statsmodels.api as sm
import geopandas as gpd
import os
import dask
import dask.dataframe as dd
import itertools
from itertools import chain
from math import sqrt, floor, ceil, isnan
import multiprocess
import multiprocessing
import importlib
from importlib import reload
from collections import Counter
from fuzzywuzzy import process, fuzz
import time
import warnings
import datetime
from datetime import datetime
from datetime import date
warnings.filterwarnings("error")

pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 400

try:
    del(FUN_proc_name)
except:
    pass
import FUN_proc_name
importlib.reload(FUN_proc_name)
from FUN_proc_name import FUN_proc_name

try:
    del(FUN_GetQ_byPlacement)
except:
    pass
import FUN_GetQ_byPlacement
importlib.reload(FUN_GetQ_byPlacement)
from FUN_GetQ_byPlacement import FUN_GetQ_byPlacement

try:
    del(FUN_GetQ_byUsageBB)
except:
    pass
import FUN_GetQ_byUsageBB
importlib.reload(FUN_GetQ_byUsageBB)
from FUN_GetQ_byUsageBB import FUN_GetQ_byUsageBB

try:
    del(FUN_GetQ_byUsageGeneral)
except:
    pass
import FUN_GetQ_byUsageGeneral
importlib.reload(FUN_GetQ_byUsageGeneral)
from FUN_GetQ_byUsageGeneral import FUN_GetQ_byUsageGeneral

try:
    del(FUN_GetQ_byUsageMain)
except:
    pass
import FUN_GetQ_byUsageMain
importlib.reload(FUN_GetQ_byUsageMain)
from FUN_GetQ_byUsageMain import FUN_GetQ_byUsageMain

try:
    del(FUN_GetQ_byIssuerType)
except:
    pass
import FUN_GetQ_byIssuerType
importlib.reload(FUN_GetQ_byIssuerType)
from FUN_GetQ_byIssuerType import FUN_GetQ_byIssuerType


# 1. SDC Global Public Finance

Notes:
- In 2020 and onwards, there are many bonds with yield below 1%, and also lower than NIC (when that field is available). Why? And is this pattern limited to very short-term bonds?
- There is a regime change in 1984: Before that NIC is more complete, but after that (including that year) price/yield is more complete. Note that for some reason, NIC is consistently higher than yield.
- SDC data on yield do not seem to be of high quality. For example,
    - For the offer by CUSIP 357010 on 07/15/2003 (https://emma.msrb.org/MS209450-MS184758-MD358401.pdf), there are multiple maturities with different interest rates, but SDC only records one of them.
    - SDC often misses bonds that repay principal ahead of time, instead of making a balloon payment at the end.

In [2]:
# %%script false --no-raise-error

###############
# Import data #
###############

%run -i SCRIPT_import_GPF.py

# Divide the "Lead Manager" field into many subfields
# Note that "Co-Managers" have other manager information, which is not utilized here
new_columns = GPF['Lead Manager'].str.split('\n', expand=True)
raw_name_GPF_colnames = ['raw_name_GPF_'+str(column) for column in new_columns.columns]
new_columns.columns = raw_name_GPF_colnames
GPF = pd.concat([GPF,new_columns],axis=1)
# Modify mistakes in sale time
threshold_date = pd.to_datetime('2050-01-01')
GPF['Sale\nDate'] = GPF['Sale\nDate'].apply(lambda x: x - pd.DateOffset(years=100) if x > threshold_date else x)
# Add a year
GPF['sale_year'] = None
GPF['sale_year'] = pd.to_datetime(GPF['Sale\nDate']).dt.year

# strip all the columns of blanks
columns = [item.strip() for item in GPF.columns]
GPF.columns = columns
# Choose columns to keep & reorder columns
GPF = GPF[[
    "Amount\n   of   \n Issue  \n($ mils)",
    "Amount\n   of   \nMaturity\n($ mils)",
    "Bid",
    "Bk \n Elig",
    "Bond\nBuyer\nUOP.1",
    "Call\nIssue",
    "County",
    "Coupon Maturity",
    "Coupon Type.1",
    "Coupon\n   of\nMaturity",
    "Financial Advisor.1",
    "Financial Advisor.2",
    "Financial\nAdvisor\nDeal(Y/N)",
    "Fitch\nInsured\nLong Term\nRating",
    "Fitch\nInsured\nShort Term\nRating",
    "General Use of Proceeds",
    "Gross\nSpread",
    "Insured\nAmount",
    "Issuer Type\nDescription",
    "Issuer",
    "Issuer\nType",
    "Lead Manager",
    "Main Use of Proceeds",
    "Maturity Amount",
    "Maturity Date",
    "Maturity",
    "Maturity\n  Year",
    "Moody's\nInsured\nLong Term\nRating",
    "Moody's\nInsured\nShort Term\nRating",
    "Net\nInterest\n  Cost",
    "Price/\n Yield\n  of\nMaturity",
    "Sale\nDate",
    "Security\n  Type",
    "State",
    "Taxable\n Code",
    "Yield Amount",
    "sale_year",
    ]+raw_name_GPF_colnames]
GPF['County_raw'] = GPF['County']
# Format county
GPF['County'] = GPF['County'].str.upper()
GPF['County'] = GPF['County'].replace(' COUNTY','')
GPF = GPF.rename(columns={
    "Amount\n   of   \n Issue  \n($ mils)":"amount",
    "Amount\n   of   \nMaturity\n($ mils)":"amount_by_maturity",
    "Bk \n Elig":"CB_Eligible",
    "Bond\nBuyer\nUOP.1":"use_of_proceeds_BB",
    "Call\nIssue":"if_callable",
    "Coupon Maturity":"TOM_coupon_rate",
    "Coupon Type.1":"coupon_type",
    "Coupon\n   of\nMaturity":"coupon_rate",
    "Financial Advisor.1":"advisor_short",
    "Financial Advisor.2":"advisor_long",
    "Financial\nAdvisor\nDeal(Y/N)":"if_advisor",
    "Fitch\nInsured\nLong Term\nRating":"Fitch_ILTR",
    "Fitch\nInsured\nShort Term\nRating":"Fitch_ISTR",
    "General Use of Proceeds":"use_of_proceeds_general",
    "Gross\nSpread":"gross_spread",
    "Insured\nAmount":"insured_amount",
    "Issuer Type\nDescription":"issuer_type_full",
    "Issuer\nType":"issuer_type",
    "Lead Manager":"lead_manager",
    "Main Use of Proceeds":"use_of_proceeds_main",
    "Maturity Amount":"TOM_amount_by_maturity",
    "Maturity Date":"TOM_maturity_date",
    "Maturity\n  Year":"maturity_date", # Somehow this rather than "Maturity" is more often non-missing
    "Moody's\nInsured\nLong Term\nRating":"Moodys_ILTR",
    "Moody's\nInsured\nShort Term\nRating":"Moodys_ISTR",
    "Net\nInterest\n  Cost":"net_interest_cost",
    "Price/\n Yield\n  of\nMaturity":"price_or_yield",
    "Sale\nDate":"sale_date",
    "Security\n  Type":"security_type",
    "Taxable\n Code":"taxable_code",
    "Yield Amount":"TOM_price_or_yield",
    })


In [3]:
##########################################
# Pre-process data for yield calculation #
##########################################

# Weighted average of yield across all maturities
GPF['avg_yield'] = None
GPF['yield_by_maturity'] = None

# Weighted average maturity
GPF['avg_maturity'] = None
GPF['all_maturity'] = None

# Amount by maturity
GPF['all_amount'] = None

# Note that coupon type is tricky: I do not have a Thompson-researched version of coupon type, so it is tricky if I want to
# have a bond-level variable "coupon type". Therefore, I retain the original "coupon_type" variable, and determine if an issue
# is "all" fixed-rate or "any" variable by checking that variable

# Whether it can be determined if the record is price or yield
GPF['IF_price_or_yield_determined'] = None
# Whether the number of tranches match in the coupon versus in the price/yield versus in the maturity field
GPF['IF_n_tranches_not_match'] = None
# Whether the yield data is collected from the "net interest cost" field. To adjust systematic differences (possibly) when 
# yield data is pulled from NIC direclty, add this as a fixed effect, and also interact it with time
GPF['IF_yield_from_NIC'] = False

GPF = GPF.reset_index(drop=True)

############################
# Handle exceptional cases #
############################

def proc_list(GPF):

    # (1) Assume that bond is sold at par if "price_or_yield" is missing, conditional on that "net interest cost" is missing. If
    # instead "net interest cost" is not missing, the priority would be to get yield from there. So should not stipulate 
    # "price_or_yield" here or the indicator variable "IF_has_price_or_yield" will be confued. Note that cases with multiple
    # tranches are also handled here. Also note that this is done for the Thompson-researched version as well

    # This step is disabled: Not sure if bond is really issued at par if there is no price/yield data. Out of conservativeness,
    # do not make the imputation
    
    if False:
        for idx,row in GPF.iterrows():
            IF_has_net_interest_cost = \
                (isinstance(row['net_interest_cost'],float) or isinstance(row['net_interest_cost'],int)) and \
                row['net_interest_cost']!=None and \
                str(row['net_interest_cost'])!='nan' and \
                str(row['net_interest_cost'])!='None' and \
                'None' not in str(row['net_interest_cost'])
            if (row['price_or_yield']==None or str(row['price_or_yield'])=='nan') and (not IF_has_net_interest_cost):
                if '\n' not in str(row['coupon_rate']):
                    GPF.loc[idx,'price_or_yield'] = 100
                else:
                    price_or_yield = ''
                    for tranch in range(0,row['coupon_rate'].count('\n')):
                        price_or_yield = price_or_yield+'100\n'
                    price_or_yield = price_or_yield+'100'
                    GPF.loc[idx,'price_or_yield'] = price_or_yield
            if (row['TOM_price_or_yield']==None or str(row['TOM_price_or_yield'])=='nan') and (not IF_has_net_interest_cost):
                if '\n' not in str(row['TOM_coupon_rate']):
                    GPF.loc[idx,'TOM_price_or_yield'] = 100
                else:
                    price_or_yield = ''
                    for tranch in range(0,row['TOM_coupon_rate'].count('\n')):
                        price_or_yield = price_or_yield+'100\n'
                    price_or_yield = price_or_yield+'100'
                    GPF.loc[idx,'TOM_price_or_yield'] = price_or_yield

    # (2) Handle zero-coupon cases. If "Zero Coupon" is one of the coupon types, there are other coupon types, and the number of
    # coupon rates is exactly "total number of maturities" minus "number of zero coupon bonds", fill in the places of zero coupon
    # bonds to have a coupon rate of 0. Note that I do not do this for the Thompson-researched version of data, as the "coupon
    # type" variable is not available there
    
    for idx,row in GPF.iterrows():
    
        coupon_rate_filled = []
        zero_coupon_idxes = []
        non_zero_coupon_idxes = []
        
        coupon_type_original = str(row['coupon_type'])
        coupon_type = str(row['coupon_type']).split('\n')
        coupon_rate = row['coupon_rate']
    
        IF_has_coupon_rate = \
            row['coupon_rate']!=None and \
            str(row['coupon_rate'])!='nan' \
            and str(row['coupon_rate'])!='None'
        
        # Handle cases where just bond
        if coupon_type_original=="Zero Coupon":
            GPF.at[idx,'coupon_rate'] = 0
            
        # When there are multiple bonds. Note that no need to handle if there is just one bond and it is not zero coupon
        elif len(coupon_type)>1 :
            # Do not handle if no zero-coupon bond
            if IF_has_coupon_rate and "Zero Coupon" in coupon_type:
                zero_coupon_idxes = [index for index,item in enumerate(coupon_type) if item=="Zero Coupon"]
                non_zero_coupon_idxes = [index for index,item in enumerate(coupon_type) if item!="Zero Coupon"]
                coupon_rate = str(coupon_rate).split('\n')
                if len(coupon_type)==len(coupon_rate)+len(zero_coupon_idxes):
                    coupon_rate_filled = [' ']*len(coupon_type)
                    physical_idx = 0
                    for sub_idx in non_zero_coupon_idxes:
                        coupon_rate_filled[sub_idx] = coupon_rate[physical_idx]
                        physical_idx = physical_idx+1
                    for sub_idx in zero_coupon_idxes:
                        coupon_rate_filled[sub_idx] = "0"
                    coupon_rate_new = coupon_rate_filled[0]
                    for item in coupon_rate_filled[1:]:
                        coupon_rate_new = coupon_rate_new+"\n"+item
                    GPF.at[idx,'coupon_rate'] = coupon_rate_new
                    
    # (3) Assume that the coupon rate applies to all maturities if there is one coupon rate but multiple tranches
    for idx,row in GPF.iterrows():
        if row['coupon_rate']!=None and str(row['coupon_rate'])!='nan':
            if '\n' in str(row['maturity_date']):
                if '\n' not in str(row['coupon_rate']):
                    coupon_rate = ''
                    for tranch in range(0,row['maturity_date'].count('\n')):
                        coupon_rate = coupon_rate+str(row['coupon_rate'])+'\n'
                    coupon_rate = coupon_rate+str(row['coupon_rate'])+'\n'
                    GPF.loc[idx,'coupon_rate'] = coupon_rate
    
    # (4) Assume that the price/yield applies to all maturities if there is one price/yield rate but multiple tranches.
    # Make this edit only if the maturities are all identical. Otherwise, it is more likely a data error and do not impute.
    for idx,row in GPF.iterrows():
        if row['price_or_yield']!=None and str(row['price_or_yield'])!='nan':
            if '\n' in str(row['maturity_date']):
                maturity_date = row['maturity_date'].split('\n')
                if_same = all(element == maturity_date[0] for element in maturity_date)
                if if_same and ('\n' not in str(row['price_or_yield'])):
                    price_or_yield = ''
                    for tranch in range(0,row['maturity_date'].count('\n')):
                        price_or_yield = price_or_yield+str(row['price_or_yield'])+'\n'
                    price_or_yield = price_or_yield+str(row['price_or_yield'])+'\n'
                    GPF.loc[idx,'price_or_yield'] = price_or_yield
    
    # (5) Take the value of "Maturity" to populate "Maturity\n  Year" (i.e., "maturity_date") if the latter is missing
    for idx,row in GPF.iterrows():
        IF_has_maturity_date = \
            row['maturity_date']!=None and \
            str(row['maturity_date'])!='nan' and \
            str(row['maturity_date'])!='None' and \
            'None' not in str(row['maturity_date']) 
        if not IF_has_maturity_date:
            GPF.at[idx,'maturity_date'] = row['Maturity']
    
    # (6) Remove if beginning or end of field is '\n'
    for idx,row in GPF.iterrows():
        if isinstance(row['price_or_yield'],str):
            if row['price_or_yield'][:1]=='\n':
                GPF.loc[idx,'price_or_yield'] = row['price_or_yield'][1:]
            if row['price_or_yield'][-1:]=='\n':
                GPF.loc[idx,'price_or_yield'] = row['price_or_yield'][:-1]
        if isinstance(row['maturity_date'],str):
            if row['maturity_date'][:1]=='\n':
                GPF.loc[idx,'maturity_date'] = row['maturity_date'][1:]
            if row['maturity_date'][-1:]=='\n':
                GPF.loc[idx,'maturity_date'] = row['maturity_date'][:-1]
        if isinstance(row['coupon_rate'],str):
            if row['coupon_rate'][:1]=='\n':
                GPF.loc[idx,'coupon_rate'] = row['coupon_rate'][1:]
            if row['coupon_rate'][-1:]=='\n':
                GPF.loc[idx,'coupon_rate'] = row['coupon_rate'][:-1]
        if isinstance(row['amount_by_maturity'],str):
            if row['amount_by_maturity'][:1]=='\n':
                GPF.loc[idx,'amount_by_maturity'] = row['amount_by_maturity'][1:]
            if row['amount_by_maturity'][-1:]=='\n':
                GPF.loc[idx,'amount_by_maturity'] = row['amount_by_maturity'][:-1]
        if isinstance(row['TOM_price_or_yield'],str):
            if row['TOM_price_or_yield'][:1]=='\n':
                GPF.loc[idx,'TOM_price_or_yield'] = row['TOM_price_or_yield'][1:]
            if row['TOM_price_or_yield'][-1:]=='\n':
                GPF.loc[idx,'TOM_price_or_yield'] = row['TOM_price_or_yield'][:-1]
        if isinstance(row['TOM_maturity_date'],str):
            if row['TOM_maturity_date'][:1]=='\n':
                GPF.loc[idx,'TOM_maturity_date'] = row['TOM_maturity_date'][1:]
            if row['TOM_maturity_date'][-1:]=='\n':
                GPF.loc[idx,'TOM_maturity_date'] = row['TOM_maturity_date'][:-1]
        if isinstance(row['TOM_coupon_rate'],str):
            if row['TOM_coupon_rate'][:1]=='\n':
                GPF.loc[idx,'TOM_coupon_rate'] = row['TOM_coupon_rate'][1:]
            if row['TOM_coupon_rate'][-1:]=='\n':
                GPF.loc[idx,'TOM_coupon_rate'] = row['TOM_coupon_rate'][:-1]
        if isinstance(row['TOM_amount_by_maturity'],str):
            if row['TOM_amount_by_maturity'][:1]=='\n':
                GPF.loc[idx,'TOM_amount_by_maturity'] = row['TOM_amount_by_maturity'][1:]
            if row['TOM_amount_by_maturity'][-1:]=='\n':
                GPF.loc[idx,'TOM_amount_by_maturity'] = row['TOM_amount_by_maturity'][:-1]
    
    # (7) Remove if beginning or end or middle of field "maturity_date" is 'None'
    for idx,row in GPF.iterrows():
        if isinstance(row['maturity_date'],str):
            if row['maturity_date'][:5]=='None\n':
                GPF.loc[idx,'maturity_date'] = row['maturity_date'][5:]
            if row['maturity_date'][-5:]=='\nNone':
                GPF.loc[idx,'maturity_date'] = row['maturity_date'][:-5]
            GPF.at[idx,'maturity_date'] = GPF.at[idx,'maturity_date'].replace('None\n','')
    
    # (8) After the prior step, there are cases where "\n" is not in "maturity_date" and "maturity_date" is a string. To avoid
    # incompatibility, convert type
    for idx,row in GPF.iterrows():
        if isinstance(row['maturity_date'],str) and '\n' not in row['maturity_date'] and\
            row['maturity_date']!=None and str(row['maturity_date'])!='nan' and str(row['maturity_date'])!='None':
            GPF.at[idx,'maturity_date'] = datetime.strptime(GPF.at[idx,'maturity_date'],"%m/%d/%y")
    
    # (9) Drop if two or three "\n" come adjacent
    for idx,row in GPF.iterrows():
        if '\n\n' in str(row['price_or_yield']):
            GPF.at[idx,'price_or_yield'] = GPF.at[idx,'price_or_yield'].replace('\n\n','\n')
        if '\n\n\n' in str(row['price_or_yield']):
            GPF.at[idx,'price_or_yield'] = GPF.at[idx,'price_or_yield'].replace('\n\n\n','\n')
        if '\n\n' in str(row['coupon_rate']):
            GPF.at[idx,'coupon_rate'] = GPF.at[idx,'coupon_rate'].replace('\n\n','\n')
        if '\n\n\n' in str(row['coupon_rate']):
            GPF.at[idx,'coupon_rate'] = GPF.at[idx,'coupon_rate'].replace('\n\n\n','\n')

    # (10) Put value of data from the non-Thompson-researched version to the Thompson-researched version, if the latter is missing.
    # For consistency, make this change for all related fields, or do not do so at all
    for idx,row in GPF.iterrows():
        if \
            (row['TOM_maturity_date']==None or \
            str(row['TOM_maturity_date'])=='nan' or \
            str(row['TOM_maturity_date'])=='None' or \
            'None' in str(row['TOM_maturity_date'])) \
            and \
            (row['TOM_coupon_rate']==None or \
            str(row['TOM_coupon_rate'])=='nan' \
            or str(row['TOM_coupon_rate'])=='None')\
            and \
            (row['TOM_price_or_yield']==None or \
            str(row['TOM_price_or_yield'])=='nan' \
            or str(row['TOM_price_or_yield'])=='None') \
            and \
            (row['TOM_amount_by_maturity']!=None or \
            str(row['TOM_amount_by_maturity'])!='nan' or \
            str(row['TOM_amount_by_maturity'])!='None' or \
            'None' not in str(row['TOM_amount_by_maturity'])):
            GPF.at[idx,'TOM_maturity_date'] = row['maturity_date']
            GPF.at[idx,'TOM_coupon_rate'] = row['coupon_rate']
            GPF.at[idx,'TOM_price_or_yield'] = row['price_or_yield']
            GPF.at[idx,'TOM_amount_by_maturity'] = row['amount_by_maturity']

    return GPF

meta_columns = list(proc_list(GPF[:10]).columns)
GPF_dd = dd.from_pandas(GPF, npartitions=20)
with dask.config.set(scheduler='processes',num_workers=20):
    GPF = GPF_dd.map_partitions(proc_list,meta=pd.DataFrame(columns=meta_columns)).compute()


In [4]:
# %%time

###############################
# Calculate yield at issuance #
###############################

def proc_list(GPF):

    # Handle case by case of each variable being missing, and within each case allow for multiple maturities
    for idx,row in GPF.iterrows():

        # Initialize variables
        IF_has_net_interest_cost = False
        IF_has_maturity_date = False
        IF_has_coupon_rate = False
        IF_has_price_or_yield = False
        IF_has_amount_by_maturity = False

        # Determine whether to use the Thompson-researched version of data. Note that below can be redundant, especially after 
        # I decide Thompson-researched version is better and put non-Thompson data to those fields if Thompson-researched data
        # are missing. I retain this structure as a legacy and because it does not create real problems
        if row['sale_year']<2003:
            COL_maturity_date = 'maturity_date'
            COL_coupon_rate = 'coupon_rate'
            COL_price_or_yield = 'price_or_yield'
            COL_amount_by_maturity = 'amount_by_maturity'
        else:
            COL_maturity_date = 'TOM_maturity_date'
            COL_coupon_rate = 'TOM_coupon_rate'
            COL_price_or_yield = 'TOM_price_or_yield'
            COL_amount_by_maturity = 'TOM_amount_by_maturity'
        
        # Whether certain fields exist
        IF_has_maturity_date = \
            row[COL_maturity_date]!=None and \
            str(row[COL_maturity_date])!='nan' and \
            str(row[COL_maturity_date])!='None' and \
            'None' not in str(row[COL_maturity_date]) 
        IF_has_coupon_rate = \
            row[COL_coupon_rate]!=None and \
            str(row[COL_coupon_rate])!='nan' \
            and str(row[COL_coupon_rate])!='None'
        IF_has_price_or_yield = \
            row[COL_price_or_yield]!=None and \
            str(row[COL_price_or_yield])!='nan' \
            and str(row[COL_price_or_yield])!='None'
        IF_has_amount_by_maturity = \
            row[COL_amount_by_maturity]!=None and \
            str(row[COL_amount_by_maturity])!='nan' and \
            str(row[COL_amount_by_maturity])!='None' and \
            'None' not in str(row[COL_amount_by_maturity]) 
        IF_has_net_interest_cost = \
            (isinstance(row['net_interest_cost'],float) or isinstance(row['net_interest_cost'],int)) and \
            row['net_interest_cost']!=None and \
            str(row['net_interest_cost'])!='nan' and \
            str(row['net_interest_cost'])!='None' and \
            'None' not in str(row['net_interest_cost'])

        # Number of entries in certain fields
        N_coupon_rate = str(row[COL_coupon_rate]).count('\n')+1
        N_price_or_yield = str(row[COL_price_or_yield]).count('\n')+1
        N_maturity_date = str(row[COL_maturity_date]).count('\n')+1
        N_amount = str(row[COL_amount_by_maturity]).count('\n')+1
        IF_num_bonds_all_consistent = \
            (N_coupon_rate==N_price_or_yield) and \
            (N_coupon_rate==N_maturity_date) and \
            (N_coupon_rate==N_amount)
        # The following indicator can be applied when coupon is not available
        IF_num_bonds_yield_mat_amt_consistent = \
            (N_price_or_yield==N_maturity_date) and \
            (N_price_or_yield==N_amount)
        # The following indicator can be applied when coupon and yield/price is not available
        IF_num_bonds_mat_amt_consistent = \
            (N_maturity_date==N_amount)

        ##########
        # Case 1 #
        ##########

        # Case 1: "coupon_rate","maturity_date","price_or_yield" are all available
        if IF_has_maturity_date and IF_has_coupon_rate and IF_has_price_or_yield and IF_has_amount_by_maturity \
            and IF_num_bonds_all_consistent:
    
            # Case 1A: If single maturity
            if N_maturity_date==1 :
                maturity = (row[COL_maturity_date]-row['sale_date']).days
                GPF.loc[idx,'avg_maturity'] = maturity
                GPF.loc[idx,'all_maturity'] = [maturity]
                GPF.loc[idx,'all_amount'] = [row[COL_amount_by_maturity]]
                # Assume that if a number is more than 80 and less than 120, it is issuing price. If less than 20, it is issuing 
                # yield. Otherwise, undetermined
                if float(row[COL_price_or_yield])<20:
                    GPF.loc[idx,'avg_yield'] = row[COL_price_or_yield]/100
                    GPF.loc[idx,'yield_by_maturity'] = [GPF.loc[idx,'avg_yield']]
                elif float(row[COL_price_or_yield])>80 and float(row[COL_price_or_yield])<120:
                    # Number of coupons to be paid
                    n_coupon = round(maturity/182)
                    GPF.loc[idx,'avg_yield'] = \
                        (1+npf.irr([-row[COL_price_or_yield]]+[row[COL_coupon_rate]/2]*(n_coupon-1)+[100+row[COL_coupon_rate]/2]))\
                        **2-1
                    GPF.loc[idx,'yield_by_maturity'] = [GPF.loc[idx,'avg_yield']]
                else:
                    GPF.loc[idx,'IF_price_or_yield_determined'] = False
    
            # Case 1B: If multiple maturity
            else:
                # If number of tranches not consistent across fields, skip
                if N_price_or_yield!=N_coupon_rate:
                    GPF.loc[idx,'IF_n_tranches_not_match'] = False
                elif N_price_or_yield!=N_maturity_date:
                    GPF.loc[idx,'IF_n_tranches_not_match'] = False
                else:
                    maturities = []
                    yields = []
                    amounts = []
                    for tranch in range(0,row[COL_coupon_rate].count('\n')+1):
                        maturity = (datetime.strptime(row[COL_maturity_date].split('\n')[tranch],"%m/%d/%y")\
                            -row['sale_date']).days
                        coupon_rate = float(row[COL_coupon_rate].split('\n')[tranch])
                        price_or_yield = float(row[COL_price_or_yield].split('\n')[tranch])
                        n_coupon = round(maturity/182)
                        amount = float(row[COL_amount_by_maturity].split('\n')[tranch].replace(',',''))
                        maturities = maturities+[maturity]
                        amounts = amounts+[amount]
                        if price_or_yield<20:
                            yields = yields+[price_or_yield/100]
                        elif price_or_yield>80 and price_or_yield<120:
                            tranch_yield = (1+npf.irr([-price_or_yield]+[coupon_rate/2]*(n_coupon-1)+[100+coupon_rate/2]))**2-1
                            yields = yields+[tranch_yield]
                        else:
                            yields = yields+[None]
                            GPF.loc[idx,'IF_price_or_yield_determined'] = False
                    if GPF.at[idx,'IF_price_or_yield_determined']!=False:
                        GPF.loc[idx,'avg_yield'] = np.dot(yields,amounts)/np.sum(amounts)
                        GPF.loc[idx,'avg_maturity'] = np.dot(maturities,amounts)/np.sum(amounts)
                    GPF.at[idx,'all_maturity'] = maturities
                    GPF.at[idx,'yield_by_maturity'] = yields
                    GPF.at[idx,'all_amount'] = amounts

        ##########
        # Case 2 #
        ##########          

        # Case 2: "coupon_rate" is not available, but "price_or_yield" and "maturity_date" is
        elif IF_has_maturity_date and (not IF_has_coupon_rate) and IF_has_price_or_yield and IF_has_amount_by_maturity \
            and IF_num_bonds_yield_mat_amt_consistent:
    
            # Case 2A: If single maturity
            if N_maturity_date==1:
                maturity = (row[COL_maturity_date]-row['sale_date']).days
                GPF.loc[idx,'avg_maturity'] = maturity
                GPF.loc[idx,'all_maturity'] = [maturity]
                GPF.loc[idx,'all_amount'] = [row[COL_amount_by_maturity]]
                # Assume that if a number is more than 80, it is issuing price. If less than 20, it is issuing yield. 
                # Otherwise, undetermined
                if float(row[COL_price_or_yield])<20:
                    GPF.loc[idx,'avg_yield'] = row[COL_price_or_yield]/100
                    GPF.loc[idx,'yield_by_maturity'] = [GPF.loc[idx,'avg_yield']]
                # Cannot do anything if coupon rate is unavailable and only price is given
                elif row[COL_price_or_yield]>80:
                    continue
    
            # Case 2B: If multiple maturity
            else:
                # If number of tranches not consistent across fields, skip
                if N_price_or_yield!=N_maturity_date:
                    GPF.loc[idx,'IF_n_tranches_not_match'] = False
                else:
                    maturities = []
                    yields = []
                    amounts = []
                    for tranch in range(0,row[COL_maturity_date].count('\n')+1):
                        maturity = (datetime.strptime(row[COL_maturity_date].split('\n')[tranch],"%m/%d/%y")-row['sale_date']).days
                        price_or_yield = float(row[COL_price_or_yield].split('\n')[tranch])
                        amount = float(row[COL_amount_by_maturity].split('\n')[tranch].replace(',',''))
                        maturities = maturities+[maturity]
                        amounts = amounts+[amount]
                        if price_or_yield<20:
                            yields = yields+[price_or_yield/100]
                        else:
                            # Note that it is impossible to calculate yield if only price is available for one tranch, or if I cannot
                            # decide whether it is price or yield
                            yields = yields+[None]
                    if len(yields)>0:
                        if None not in yields:
                            GPF.loc[idx,'avg_yield'] = np.dot(yields,amounts)/np.sum(amounts)
                        else:
                            GPF.loc[idx,'avg_yield'] = None
                        GPF.loc[idx,'avg_maturity'] = np.dot(maturities,amounts)/np.sum(amounts)
                    GPF.at[idx,'all_maturity'] = maturities
                    GPF.at[idx,'yield_by_maturity'] = yields
                    GPF.at[idx,'all_amount'] = amounts

        ##########
        # Case 3 #
        ##########

        # Case 3: When "net interest cost" is available, use it directly and do not compute yield myself
        # Note that "net interest cost" tend to be populated in 1983 and before
        elif IF_has_maturity_date and IF_has_net_interest_cost and IF_has_amount_by_maturity \
            and IF_num_bonds_mat_amt_consistent:
            GPF.loc[idx,'IF_yield_from_NIC'] = True
            GPF.loc[idx,'avg_yield'] = row['net_interest_cost']/100
            # If single maturity
            if N_maturity_date==1:
                maturity = (row[COL_maturity_date]-row['sale_date']).days
                GPF.loc[idx,'avg_maturity'] = maturity
                GPF.at[idx,'all_maturity'] = [maturity]
                GPF.loc[idx,'all_amount'] = [row[COL_amount_by_maturity]]
            # If multiple maturity
            else:
                maturities = []
                amounts = []
                for tranch in range(0,row[COL_maturity_date].count('\n')+1):
                    maturity = (datetime.strptime(row[COL_maturity_date].split('\n')[tranch],"%m/%d/%y")-row['sale_date']).days
                    amount = float(row[COL_amount_by_maturity].split('\n')[tranch].replace(',',''))
                    maturities = maturities+[maturity]
                    amounts = amounts+[amount]
                if None not in maturities:
                    GPF.loc[idx,'avg_maturity'] = np.mean(maturities)
                else:
                    GPF.loc[idx,'avg_maturity'] = None
                GPF.at[idx,'all_maturity'] = maturities
                GPF.at[idx,'all_amount'] = amounts
            continue
    
    
    return GPF

meta_columns = list(proc_list(GPF[:10]).columns)
GPF_dd = dd.from_pandas(GPF, npartitions=20)
with dask.config.set(scheduler='processes',num_workers=20):
    GPF = GPF_dd.map_partitions(proc_list,meta=pd.DataFrame(columns=meta_columns)).compute()

# Reorder columns
first_columns = ['sale_date','net_interest_cost',
    'maturity_date','coupon_rate','price_or_yield','amount_by_maturity',
    'TOM_maturity_date','TOM_coupon_rate','TOM_price_or_yield','TOM_amount_by_maturity',
    'avg_yield','avg_maturity','yield_by_maturity','all_maturity','all_amount',
    'IF_price_or_yield_determined']
GPF = GPF[first_columns+sorted([item for item in GPF.columns if item not in first_columns])]


In [5]:
########################################################
# Calculate yield of synthetic risk-free treasury bond #
########################################################

# Notes:
# (1) Without coupon rate, it is impossible to calculate the price (yield) of the synthetic treasury bond. For example, consider 
# Bond A: Pays $5 one year from now, and $5 two years from now, and Bond B: Pays $100*(1+5%)^2 two years from now. These two have
# the same yield. Suppose that treasury yield in one year is 0% and in two years is 20%. Then price of first synthetic bond is
# higher than the second, and the yield of the first synthetic bond is lower than the second. In other words, without coupon rate,
# I do not know when the cash flow is going to come, so I do not know what is the component of risk-free rate that I should tease
# out from the return of the municipal bond yield. An INACCURATE approximation can be simply using yield of municipal bond minus 
# that of treasury bond, but it is erraneous to do so.
# 
# Luckily except for later parts of the sample, coupon rate is usually available.

def proc_list(GPF):

    GPF = GPF.copy()
    
    # Treasury yield
    feds200628 = pd.read_csv("../RawData/FedBOG/feds200628.csv", header=9)
    feds200628 = feds200628[~pd.isnull(feds200628['SVENY01'])]
    columns = ['Date']+ \
        ['SVENY0'+str(i) for i in range(1,10)]+ \
        ['SVENY'+str(i) for i in range(10,31)]
    feds200628 = feds200628[columns]
    new_columns = ['Date']+ \
        ['SVENY'+str(i) for i in range(1,10)]+ \
        ['SVENY'+str(i) for i in range(10,31)]
    feds200628.columns = new_columns
    feds200628['Date'] = pd.to_datetime(feds200628['Date'])
    threshold_date = pd.to_datetime('2050-01-01')
    feds200628['Date'] = feds200628['Date'].apply(lambda x: x - pd.DateOffset(years=100) if x > threshold_date else x)
    
    max_year_7 = [pd.Timestamp(1961,6,14,0,0,0),pd.Timestamp(1971,8,15,0,0,0)]
    max_year_10 = [pd.Timestamp(1971,8,16,0,0,0),pd.Timestamp(1971,11,14,0,0,0)]
    max_year_15 = [pd.Timestamp(1971,11,15,0,0,0),pd.Timestamp(1981,7,1,0,0,0)]
    max_year_20 = [pd.Timestamp(1981,7,2,0,0,0),pd.Timestamp(1985,11,24,0,0,0)]
    max_year_30 = [pd.Timestamp(1985,11,25,0,0,0),pd.Timestamp(2023,11,3,0,0,0)]
    
    GPF['sync_bond_yield_by_maturity'] = None
    
    for idx,row in GPF.iterrows():

        # Initialize variables
        IF_has_net_interest_cost = False
        IF_has_maturity_date = False
        IF_has_coupon_rate = False
        IF_has_price_or_yield = False
        IF_has_amount_by_maturity = False
        
        # Determine whether to use the Thompson-researched version of data
        if row['sale_year']<2003:
            COL_maturity_date = 'maturity_date'
            COL_coupon_rate = 'coupon_rate'
            COL_price_or_yield = 'price_or_yield'
            COL_amount_by_maturity = 'amount_by_maturity'
        else:
            COL_maturity_date = 'TOM_maturity_date'
            COL_coupon_rate = 'TOM_coupon_rate'
            COL_price_or_yield = 'TOM_price_or_yield'
            COL_amount_by_maturity = 'TOM_amount_by_maturity'
    
        # Continue if coupon rate is missing
        IF_has_coupon_rate = \
            row[COL_coupon_rate]!=None and \
            str(row[COL_coupon_rate])!='nan' \
            and str(row[COL_coupon_rate])!='None'
        IF_has_maturity_date = \
            row[COL_maturity_date]!=None and \
            str(row[COL_maturity_date])!='nan' and \
            str(row[COL_maturity_date])!='None' and \
            'None' not in str(row[COL_maturity_date]) 
        IF_has_price_or_yield = \
            row[COL_price_or_yield]!=None and \
            str(row[COL_price_or_yield])!='nan' \
            and str(row[COL_price_or_yield])!='None'
        IF_has_amount_by_maturity = \
            row[COL_amount_by_maturity]!=None and \
            str(row[COL_amount_by_maturity])!='nan' and \
            str(row[COL_amount_by_maturity])!='None' and \
            'None' not in str(row[COL_amount_by_maturity]) 
        if (not IF_has_coupon_rate) or (not IF_has_maturity_date) or (not IF_has_price_or_yield) or (not IF_has_amount_by_maturity):
            continue
    
        # Number of entries in certain fields
        N_coupon_rate = str(row[COL_coupon_rate]).count('\n')+1
        N_price_or_yield = str(row[COL_price_or_yield]).count('\n')+1
        N_maturity_date = str(row[COL_maturity_date]).count('\n')+1
        N_amount = str(row[COL_amount_by_maturity]).count('\n')+1
        IF_num_bonds_all_consistent = \
            (N_coupon_rate==N_price_or_yield) and \
            (N_coupon_rate==N_maturity_date) and \
            (N_coupon_rate==N_amount)
    
        # Obtain the treasury zero-coupon yield curve at the closest date
        feds200628_copy = feds200628.copy()
        feds200628_copy['dif_date'] = np.abs(feds200628_copy['Date']-row['sale_date'])
        feds200628_copy = feds200628_copy.sort_values('dif_date').reset_index()
    
        sync_bond_yield_by_maturity = []
    
        # If single maturity
        if N_coupon_rate==1 and IF_num_bonds_all_consistent:
    
            if row[COL_price_or_yield]>20 and row[COL_price_or_yield]<80:
                continue
            
            coupon_rate = float(row[COL_coupon_rate])
            maturity = row['all_maturity'][0]
    
            cf = []
            discount_factor = []
            N_coupons = int(np.max([1,np.around(maturity/(365/2))]))
    
            # Determine if synthetic bond can be constructed. Cannot do so if the length of zero-coupon yields is not long enough
            rf_available = False
            if row['sale_date']>max_year_7[0] and row['sale_date']<=max_year_7[1] and maturity<=7*365:
                rf_available = True
            if row['sale_date']>max_year_10[0] and row['sale_date']<=max_year_10[1] and maturity<=10*365:
                rf_available = True
            if row['sale_date']>max_year_15[0] and row['sale_date']<=max_year_15[1] and maturity<=15*365:
                rf_available = True
            if row['sale_date']>max_year_20[0] and row['sale_date']<=max_year_20[1] and maturity<=20*365:
                rf_available = True
            if row['sale_date']>max_year_30[0] and row['sale_date']<=max_year_30[1] and maturity<=30*365:
                rf_available = True
    
            if rf_available:
                # Construct a series of cash flow for each bond
                for cf_idx in range(0,N_coupons):
                    cf = cf+[coupon_rate/2]
                cf[N_coupons-1] = cf[N_coupons-1]+100
        
                # Construct a series of discount factor for each bond
                for cf_idx in range(0,N_coupons):
                    if cf_idx==0:
                        discount_factor = discount_factor+[feds200628_copy['SVENY1'][0]]
                    elif cf_idx%2==1:
                        discount_factor = discount_factor+[feds200628_copy['SVENY'+str(ceil(cf_idx/2))][0]]
                    elif cf_idx%2==0:
                        discount_factor = discount_factor+\
                            [(feds200628_copy['SVENY'+str(ceil(cf_idx/2))][0]
                            +feds200628_copy['SVENY'+str(ceil(cf_idx/2)+1)][0])/2]
                discount_factor = [(1/(1+discount_factor[disc_idx]/100))**((disc_idx+1)/2) for disc_idx in range(0,N_coupons)]
    
                # Bond price and yield of synthetic bond
                sync_bond_price = np.sum(np.dot(cf,discount_factor))
                cf = [-sync_bond_price]+cf
                sync_bond_yield = (1+npf.irr(cf))**2-1
                sync_bond_yield_by_maturity = sync_bond_yield_by_maturity+[sync_bond_yield]
    
                # Record data
                GPF.at[idx,'sync_bond_yield_by_maturity'] = sync_bond_yield_by_maturity
    
    
        # If multiple maturity, go over bond by bond
        if N_coupon_rate>1 and IF_num_bonds_all_consistent:
    
            for bond_idx in range(0,N_maturity_date):
    
                if (float(row[COL_price_or_yield].split('\n')[bond_idx])>20) and \
                    (float(row[COL_price_or_yield].split('\n')[bond_idx])<80):
                    sync_bond_yield_by_maturity = sync_bond_yield_by_maturity+[None]
                    continue
    
                coupon_rate = float(row[COL_coupon_rate].split('\n')[bond_idx])
                maturity = row['all_maturity'][bond_idx]
        
                cf = []
                discount_factor = []
                N_coupons = int(np.max([1,np.around(maturity/(365/2))]))
        
                # Determine if synthetic bond can be constructed. Cannot do so if the length of zero-coupon yields is not long enough
                rf_available = False
                if row['sale_date']>max_year_7[0] and row['sale_date']<=max_year_7[1] and maturity<=7*365:
                    rf_available = True
                if row['sale_date']>max_year_10[0] and row['sale_date']<=max_year_10[1] and maturity<=10*365:
                    rf_available = True
                if row['sale_date']>max_year_15[0] and row['sale_date']<=max_year_15[1] and maturity<=15*365:
                    rf_available = True
                if row['sale_date']>max_year_20[0] and row['sale_date']<=max_year_20[1] and maturity<=20*365:
                    rf_available = True
                if row['sale_date']>max_year_30[0] and row['sale_date']<=max_year_30[1] and maturity<=30*365:
                    rf_available = True
        
                if rf_available:
                    # Construct a series of cash flow for each bond
                    for cf_idx in range(0,N_coupons):
                        cf = cf+[coupon_rate/2]
                    cf[N_coupons-1] = cf[N_coupons-1]+100
            
                    # Construct a series of discount factor for each bond
                    for cf_idx in range(0,N_coupons):
                        if cf_idx==0:
                            discount_factor = discount_factor+[feds200628_copy['SVENY1'][0]]
                        elif cf_idx%2==1:
                            discount_factor = discount_factor+[feds200628_copy['SVENY'+str(ceil(cf_idx/2))][0]]
                        elif cf_idx%2==0:
                            discount_factor = discount_factor+\
                                [(feds200628_copy['SVENY'+str(ceil(cf_idx/2))][0]
                                +feds200628_copy['SVENY'+str(ceil(cf_idx/2)+1)][0])/2]
                    discount_factor = [(1/(1+discount_factor[disc_idx]/100))**((disc_idx+1)/2) for disc_idx in range(0,N_coupons)]
        
                    # Bond price and yield of synthetic bond
                    sync_bond_price = np.sum(np.dot(cf,discount_factor))
                    cf = [-sync_bond_price]+cf
                    sync_bond_yield = (1+npf.irr(cf))**2-1
                    sync_bond_yield_by_maturity = sync_bond_yield_by_maturity+[sync_bond_yield]
                else:
                    sync_bond_yield_by_maturity = sync_bond_yield_by_maturity+[None]
        
            GPF.at[idx,'sync_bond_yield_by_maturity'] = sync_bond_yield_by_maturity

    return GPF



meta_columns = list(proc_list(GPF[:10]).columns)
GPF_dd = dd.from_pandas(GPF, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    GPF = GPF_dd.map_partitions(proc_list,meta=pd.DataFrame(columns=meta_columns)).compute()


In [6]:
####################
# Calculate spread #
####################

# Calculate spread. Note that I calculate spread a bit differently from Li and Zhu: Theirs is from the perspective of a taxed
# individual, while mine is from the perspective of a non-taxed individual

tax_rate = {
    1967:0.700,1968:0.700,1969:0.700,1970:0.700,1971:0.700,1972:0.700,1973:0.700,
    1974:0.700,1975:0.700,1976:0.700,1977:0.700,1978:0.700,1979:0.700,1980:0.700,
    1981:0.700,1982:0.500,1983:0.500,1984:0.500,1985:0.500,1986:0.500,1987:0.385,
    1988:0.280,1989:0.280,1990:0.280,1991:0.310,1992:0.310,1993:0.396,1994:0.396,
    1995:0.396,1996:0.396,1997:0.396,1998:0.396,1999:0.396,2000:0.396,2001:0.391,
    2002:0.386,2003:0.350,2004:0.350,2005:0.350,2006:0.350,2007:0.350,2008:0.350,
    2009:0.350,2010:0.350,2011:0.350,2012:0.350,2013:0.396,2014:0.396,2015:0.396,
    2016:0.396,2017:0.396,2018:0.370,2019:0.370,2020:0.370,2021:0.370,2022:0.370,
    2023:0.370,
    }

GPF = GPF.reset_index(drop=True)
GPF['spread_by_maturity'] = None
GPF['avg_spread'] = None

def proc_list(GPF):

    for idx,row in GPF.iterrows():
        
        spread_by_maturity = []
        if row['sync_bond_yield_by_maturity']!=None and row['yield_by_maturity']!=None:
            for bond_idx in range(0,len(GPF.at[idx,'sync_bond_yield_by_maturity'])):
                if row['sync_bond_yield_by_maturity'][bond_idx]==None or \
                    row['yield_by_maturity'][bond_idx]==None:
                    spread_by_maturity = spread_by_maturity+[None]
                else:
                    # Adjust for tax here
                    if row['taxable_code']=='E':
                        spread_by_maturity = spread_by_maturity+\
                            [row['yield_by_maturity'][bond_idx]
                            -row['sync_bond_yield_by_maturity'][bond_idx]*(1-tax_rate[row['sale_year']])]
                    elif row['taxable_code']=='A' or 'T':
                        spread_by_maturity = spread_by_maturity+\
                            [row['yield_by_maturity'][bond_idx]*(1-tax_rate[row['sale_year']])
                            -row['sync_bond_yield_by_maturity'][bond_idx]*(1-tax_rate[row['sale_year']])]
            GPF.at[idx,'spread_by_maturity'] = spread_by_maturity

        if GPF.at[idx,'spread_by_maturity']!=None:
            if None not in GPF.at[idx,'spread_by_maturity']:
                GPF.loc[idx,'avg_spread'] = \
                    np.dot(GPF.at[idx,'spread_by_maturity'],row['all_amount'])/np.sum(row['all_amount'])

    return GPF

meta_columns = list(proc_list(GPF[:10]).columns)
GPF_dd = dd.from_pandas(GPF, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    GPF = GPF_dd.map_partitions(proc_list,meta=pd.DataFrame(columns=meta_columns)).compute()


In [7]:
###########################
# Export issue-level data #
###########################

GPF.to_csv("../RawData/SDC/GPF.csv")

In [8]:
%%time

###################################
# Quantitiy of debt, county-level #
###################################

# Obtain quantity of debt at county level, by aggregate and also by 
# (1) The method of placement
# (2) The use of proceeds
# (3) Type of borrowing entity

# Note that should consider taking log when using this variable

# For speed reasons, proceed year by year
Years = list(range(1967,2023))

GPFAmount = GPF[['State','County','sale_year','amount',
    'issuer_type_full','Bid','use_of_proceeds_BB','use_of_proceeds_general','use_of_proceeds_main']].copy()

GPFAmount = GPFAmount[GPFAmount['State']!='nan']
GPFAmount = GPFAmount[GPFAmount['State']!='AS']
GPFAmount = GPFAmount[GPFAmount['State']!='DC']
GPFAmount = GPFAmount[GPFAmount['State']!='FF']
GPFAmount = GPFAmount[GPFAmount['State']!='GU']
GPFAmount = GPFAmount[GPFAmount['State']!='MR']
GPFAmount = GPFAmount[GPFAmount['State']!='PR']
GPFAmount = GPFAmount[GPFAmount['State']!='TT']
GPFAmount = GPFAmount[GPFAmount['State']!='VI']

GPFAmount = GPFAmount.reset_index(drop=True)
GPFAmount = GPFAmount[~pd.isnull(GPFAmount['County'])]

def proc_list(GPFAmount):
    GPFAmount = GPFAmount.copy()
    GPFAmount['County'] = GPFAmount['County'].str.replace(' AND ','/')
    GPFAmount_New = []
    for idx,row in GPFAmount.iterrows():
        if '/' not in row['County']:
            GPFAmount_New = GPFAmount_New+[dict(row)]
        else:
            Countys = row['County'].split('/')
            for County in Countys:
                row_new = dict(row)
                row_new['County'] = County
                row_new['amount'] = row['amount']/len(Countys)
                GPFAmount_New = GPFAmount_New+[row_new]
    GPFAmount_New = pd.DataFrame(GPFAmount_New)
    GPFAmount_New['County'] = GPFAmount_New['County'].str.strip()
    return GPFAmount_New

meta_columns = list(proc_list(GPFAmount[:10]).columns)
GPFAmount_dd = dd.from_pandas(GPFAmount, npartitions=40)
with dask.config.set(scheduler='processes',num_workers=40):
    GPFAmount = GPFAmount_dd.map_partitions(proc_list,meta=pd.DataFrame(columns=meta_columns)).compute()

#---------------------#
# Method of placement #
#---------------------#

input_list = [(year,GPFAmount) for year in Years]
if __name__ == '__main__':
    with multiprocessing.Pool(processes = 10) as p:
        StateXCountyXBid = p.starmap(FUN_GetQ_byPlacement, input_list)
StateXCountyXBid = pd.concat(StateXCountyXBid)
StateXCountyXBid.to_parquet("../CleanData/SDC/StateXCountyXBid.parquet")

#-----------------#
# Use of proceeds #
#-----------------#

input_list = [(year,GPFAmount) for year in Years]
if __name__ == '__main__':
    with multiprocessing.Pool(processes = 10) as p:
        StateXCountyXUsageBB = p.starmap(FUN_GetQ_byUsageBB, input_list)
StateXCountyXUsageBB = pd.concat(StateXCountyXUsageBB)
StateXCountyXUsageBB.to_parquet("../CleanData/SDC/StateXCountyXUsageBB.parquet")

input_list = [(year,GPFAmount) for year in Years]
if __name__ == '__main__':
    with multiprocessing.Pool(processes = 10) as p:
        StateXCountyXUsageGeneral = p.starmap(FUN_GetQ_byUsageGeneral, input_list)
StateXCountyXUsageGeneral = pd.concat(StateXCountyXUsageGeneral)
StateXCountyXUsageGeneral.to_parquet("../CleanData/SDC/StateXCountyXUsageGeneral.parquet")

input_list = [(year,GPFAmount) for year in Years]
if __name__ == '__main__':
    with multiprocessing.Pool(processes = 10) as p:
        StateXCountyXUsageMain = p.starmap(FUN_GetQ_byUsageMain, input_list)
StateXCountyXUsageMain = pd.concat(StateXCountyXUsageMain)
StateXCountyXUsageMain.to_parquet("../CleanData/SDC/StateXCountyXUsageMain.parquet")

#----------------#
# Type of issuer #
#----------------#

input_list = [(year,GPFAmount) for year in Years]
if __name__ == '__main__':
    with multiprocessing.Pool(processes = 10) as p:
        StateXCountyXIssuerType = p.starmap(FUN_GetQ_byIssuerType, input_list)
StateXCountyXIssuerType = pd.concat(StateXCountyXIssuerType)
StateXCountyXIssuerType.to_parquet("../CleanData/SDC/StateXCountyXIssuerType.parquet")


CPU times: user 29.9 s, sys: 9.15 s, total: 39 s
Wall time: 1h 1min 53s


# 2. SDC M&A

In [400]:
# Note that 
# (1) In the data source I have all M&As in the financial industry since 1990. Consider expanding the dataset?
# (2) This dataset also has withdrawn mergers, which is dropped when requiring new shares > 50%. But, maybe partial ownership can 
# affect market power as well? Sample size significantly increases when this restriction is removed

MA_Fin_19800101_19861231 = pd.read_excel("../RawData/SDC/M&A_Fin_19800101_19861231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_19870101_19891231 = pd.read_excel("../RawData/SDC/M&A_Fin_19870101_19891231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_19900101_19951231 = pd.read_excel("../RawData/SDC/M&A_Fin_19900101_19951231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_19960101_19981231 = pd.read_excel("../RawData/SDC/M&A_Fin_19960101_19981231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_19990101_20011231 = pd.read_excel("../RawData/SDC/M&A_Fin_19990101_20011231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20020101_20041231 = pd.read_excel("../RawData/SDC/M&A_Fin_20020101_20041231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20050101_20061231 = pd.read_excel("../RawData/SDC/M&A_Fin_20050101_20061231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20070101_20091231 = pd.read_excel("../RawData/SDC/M&A_Fin_20070101_20091231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20100101_20121231 = pd.read_excel("../RawData/SDC/M&A_Fin_20100101_20121231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20130101_20151231 = pd.read_excel("../RawData/SDC/M&A_Fin_20130101_20151231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20160101_20181231 = pd.read_excel("../RawData/SDC/M&A_Fin_20160101_20181231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20190101_20211231 = pd.read_excel("../RawData/SDC/M&A_Fin_20190101_20211231.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA_Fin_20220101_20230930 = pd.read_excel("../RawData/SDC/M&A_Fin_20220101_20230930.xlsx",skiprows=[0],
    parse_dates=[' Rank Date','  Date\nAnnounced','  Date\nEffective','Date\nEffective/\nUnconditional','  Date\nWithdrawn'],
    dtype={'Acquiror Name':str,'Target Name':str})
MA = pd.concat([
    MA_Fin_19800101_19861231,
    MA_Fin_19870101_19891231,
    MA_Fin_19900101_19951231,
    MA_Fin_19960101_19981231,
    MA_Fin_19990101_20011231,
    MA_Fin_20020101_20041231,
    MA_Fin_20050101_20061231,
    MA_Fin_20070101_20091231,
    MA_Fin_20100101_20121231,
    MA_Fin_20130101_20151231,
    MA_Fin_20160101_20181231,
    MA_Fin_20190101_20211231,
    MA_Fin_20220101_20230930,
])

# Rename variable
MA = MA.rename(columns={
    '  Date\nAnnounced':'date_announced',
    '  Date\nEffective':'date_effective',
    'Target Name':'target_raw',
    'Acquiror Name':'acquiror_raw',
    '  %\nOwned\nAfter\nTrans-\naction':'new_share'})
MA = MA[['date_announced','date_effective','target_raw','acquiror_raw','new_share','Synopsis','Status']]
# Clean names
MA['target'] = MA['target_raw'].apply(FUN_proc_name)
MA['acquiror'] = MA['acquiror_raw'].apply(FUN_proc_name)
MA['sale_year'] = pd.to_datetime(MA['date_effective']).dt.year
MA['announce_year'] = pd.to_datetime(MA['date_announced']).dt.year

# Export data
MA.to_csv("../RawData/SDC/MA.csv")

# 3. Demographics

For population, use Census Bureau data for 2021-2022, but NIH data piror to that, as some files are missing from Census Bureau data.

In [10]:
#############################
# CSA-level population size #
#############################

# Data for 2021-2022
CSA_POP_2022 = pd.read_csv("../RawData/MSA/POP/csa-est2022.csv",encoding = "ISO-8859-1")
CSA_POP_2022 = CSA_POP_2022[CSA_POP_2022['LSAD']=='Combined Statistical Area']
CSA_POP_2022 = CSA_POP_2022[['CSA','POPESTIMATE2021','POPESTIMATE2022']].reset_index(drop=True)
CSA_POP_2022 = CSA_POP_2022.rename(columns={'CSA':'CSA Code','POPESTIMATE2021':'pop_2021','POPESTIMATE2022':'pop_2022'})

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})
us1969_2020 = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
us1969_2020 = us1969_2020.reset_index()

CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]

pop_by_CSA = us1969_2020.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_CSA = pop_by_CSA.groupby(['year','CSA Code']).agg({'pop':sum}).reset_index()

# Combine two data sources
pop_by_CSA_2021 = CSA_POP_2022[['CSA Code','pop_2021']].rename(columns={'pop_2021':'pop'})
pop_by_CSA_2021['year'] = 2021
pop_by_CSA_2022 = CSA_POP_2022[['CSA Code','pop_2022']].rename(columns={'pop_2022':'pop'})
pop_by_CSA_2022['year'] = 2022
# Stipulate population of 2023 to be identical as 2022
pop_by_CSA_2023 = pop_by_CSA_2022.copy()
pop_by_CSA_2023['year'] = 2023
pop_by_CSA = pd.concat([pop_by_CSA,pop_by_CSA_2021,pop_by_CSA_2022,pop_by_CSA_2023])

pop_by_CSA.to_csv("../RawData/MSA/POP/CSA_POP.csv")

In [11]:
##############################
# CBSA-level population size #
##############################

# "CSA_POP_2022" do not have all the CBSAs. Use county level data from "us1969_2020" and stipulate for years after 2020

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})
us1969_2020 = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
us1969_2020 = us1969_2020.reset_index()

CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]

pop_by_CBSA = us1969_2020.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_CBSA = pop_by_CBSA.groupby(['year','CBSA Code']).agg({'pop':sum}).reset_index()

pop_by_CBSA['year'] = pop_by_CBSA['year'].astype(int)

# Combine two data sources
pop_by_CBSA_2021 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2021['year'] = 2021
pop_by_CBSA_2022 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2022['year'] = 2022
pop_by_CBSA_2023 = pop_by_CBSA[pop_by_CBSA['year']==2020].copy()
pop_by_CBSA_2023['year'] = 2023
pop_by_CBSA = pd.concat([pop_by_CBSA,pop_by_CBSA_2021,pop_by_CBSA_2022,pop_by_CBSA_2023])

pop_by_CBSA.to_csv("../RawData/MSA/POP/CBSA_POP.csv")

In [13]:
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]
CBSAData['FIPS State Code'] = CBSAData['FIPS State Code'].astype(int).astype(str)
CBSAData['FIPS County Code'] = CBSAData['FIPS County Code'].astype(int).astype(str)
CBSAData.loc[CBSAData['FIPS State Code'].str.len()==1,'FIPS State Code'] = '0'+CBSAData['FIPS State Code'][CBSAData['FIPS State Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==1,'FIPS County Code'] = '00'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==1]
CBSAData.loc[CBSAData['FIPS County Code'].str.len()==2,'FIPS County Code'] = '0'+CBSAData['FIPS County Code'][CBSAData['FIPS County Code'].str.len()==2]


For income, use data from BEA.

In [15]:
####################
# CSA-level income #
####################

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
pop_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Population (persons) 1/'].copy()
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace(' ','')
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace('"','')
pop_by_county['FIPS State Code'] = pop_by_county['GeoFIPS'].str[:2]
pop_by_county['FIPS County Code'] = pop_by_county['GeoFIPS'].str[2:5]


inc_by_county = inc_by_county.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_county = pop_by_county.merge(CBSAData[~pd.isnull(CBSAData['CSA Code'])][['CSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])

inc_by_CSA = pd.DataFrame()

for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)

    pop_by_county_oneyear = pop_by_county[[str(year),'CSA Code','FIPS State Code','FIPS County Code']]
    pop_by_county_oneyear = pop_by_county_oneyear.rename(columns={str(year):'pop'})
    pop_by_county_oneyear = pop_by_county_oneyear[pop_by_county_oneyear['pop']!='(NA)']
    pop_by_county_oneyear['pop'] = pop_by_county_oneyear['pop'].astype(float)
    
    inc_by_county_oneyear = inc_by_county_oneyear.merge(pop_by_county_oneyear,on=['FIPS State Code','FIPS County Code','CSA Code'])

    # Calculate weighted average income
    inc_by_county_oneyear['incXpop'] = inc_by_county_oneyear['inc']*inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear.groupby(['CSA Code']).agg({'pop':sum,'incXpop':sum})
    inc_by_county_oneyear = inc_by_county_oneyear.reset_index()
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['incXpop']/inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear[['inc','CSA Code']]
    inc_by_county_oneyear['year'] = year

    inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])

# Supplement with 2022 and 2023, for which I assume income to be the same as 2021
inc_by_county_oneyear['year'] = 2022
inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])
inc_by_county_oneyear['year'] = 2023
inc_by_CSA = pd.concat([inc_by_CSA,inc_by_county_oneyear])

inc_by_CSA.to_csv("../RawData/MSA/CAINC1/CSA_INC.csv")

In [ ]:
#####################
# CBSA-level income #
#####################

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
pop_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Population (persons) 1/'].copy()
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace(' ','')
pop_by_county['GeoFIPS'] = pop_by_county['GeoFIPS'].str.replace('"','')
pop_by_county['FIPS State Code'] = pop_by_county['GeoFIPS'].str[:2]
pop_by_county['FIPS County Code'] = pop_by_county['GeoFIPS'].str[2:5]


inc_by_county = inc_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])
pop_by_county = pop_by_county.merge(CBSAData[~pd.isnull(CBSAData['CBSA Code'])][['CBSA Code','FIPS State Code','FIPS County Code']],
    on=['FIPS State Code','FIPS County Code'])

inc_by_CBSA = pd.DataFrame()

for year in range(1969,2022):
    
    inc_by_county_oneyear = inc_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    inc_by_county_oneyear = inc_by_county_oneyear.rename(columns={str(year):'inc'})
    inc_by_county_oneyear = inc_by_county_oneyear[inc_by_county_oneyear['inc']!='(NA)']
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['inc'].astype(float)

    pop_by_county_oneyear = pop_by_county[[str(year),'CBSA Code','FIPS State Code','FIPS County Code']]
    pop_by_county_oneyear = pop_by_county_oneyear.rename(columns={str(year):'pop'})
    pop_by_county_oneyear = pop_by_county_oneyear[pop_by_county_oneyear['pop']!='(NA)']
    pop_by_county_oneyear['pop'] = pop_by_county_oneyear['pop'].astype(float)
    
    inc_by_county_oneyear = inc_by_county_oneyear.merge(pop_by_county_oneyear,on=['FIPS State Code','FIPS County Code','CBSA Code'])

    # Calculate weighted average income
    inc_by_county_oneyear['incXpop'] = inc_by_county_oneyear['inc']*inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear.groupby(['CBSA Code']).agg({'pop':sum,'incXpop':sum})
    inc_by_county_oneyear = inc_by_county_oneyear.reset_index()
    inc_by_county_oneyear['inc'] = inc_by_county_oneyear['incXpop']/inc_by_county_oneyear['pop']
    inc_by_county_oneyear = inc_by_county_oneyear[['inc','CBSA Code']]
    inc_by_county_oneyear['year'] = year

    inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])

# Supplement with 2022 and 2023, for which I assume income to be the same as 2021
inc_by_county_oneyear['year'] = 2022
inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])
inc_by_county_oneyear['year'] = 2023
inc_by_CBSA = pd.concat([inc_by_CBSA,inc_by_county_oneyear])

inc_by_CBSA.to_csv("../RawData/MSA/CAINC1/CBSA_INC.csv")

In [150]:
############################################
# County-level data, including black ratio #
############################################

# Data prior to that
us1969_2020 = pd.read_csv("../RawData/MSA/POP/us.1969_2020.19ages.adjusted.txt",header=None)

us1969_2020['year'] = us1969_2020[0].str.slice(0,4)
us1969_2020['state'] = us1969_2020[0].str.slice(4,6)
us1969_2020['state_FIPS'] = us1969_2020[0].str.slice(6,8)
us1969_2020['county_FIPS'] = us1969_2020[0].str.slice(8,11)
us1969_2020['registry'] = us1969_2020[0].str.slice(11,13)
us1969_2020['race'] = us1969_2020[0].str.slice(13,14)
us1969_2020['origin'] = us1969_2020[0].str.slice(14,15)
us1969_2020['sex'] = us1969_2020[0].str.slice(15,16)
us1969_2020['age'] = us1969_2020[0].str.slice(16,18)
us1969_2020['pop'] = us1969_2020[0].str.slice(18,26).astype(int)
us1969_2020 = us1969_2020.drop(columns=[0])

us1969_2020 = us1969_2020.rename(columns={'state_FIPS':'FIPS State Code','county_FIPS':'FIPS County Code'})

# Calculate black ratio
black_pop = us1969_2020[us1969_2020['race']=='2'][['year','FIPS State Code','FIPS County Code','pop']]
black_pop = black_pop.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
black_pop = black_pop.rename(columns={'pop':'black_pop'})

county_pop = us1969_2020.groupby(['FIPS State Code','FIPS County Code','year']).agg({'pop':sum})
county_pop = county_pop.reset_index()

black_pop = black_pop.merge(county_pop,on=['FIPS State Code','FIPS County Code','year'])
black_pop['black_ratio'] = black_pop['black_pop']/black_pop['pop']

# Income by county, including those not in CBSA
CAINC1__ALL_AREAS_1969_2021 = pd.read_csv("../RawData/MSA/CAINC1/CAINC1__ALL_AREAS_1969_2021.csv",encoding = "ISO-8859-1")
inc_by_county = CAINC1__ALL_AREAS_1969_2021[CAINC1__ALL_AREAS_1969_2021['Description']=='Per capita personal income (dollars) 2/'].copy()
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace(' ','')
inc_by_county['GeoFIPS'] = inc_by_county['GeoFIPS'].str.replace('"','')
inc_by_county['FIPS State Code'] = inc_by_county['GeoFIPS'].str[:2]
inc_by_county['FIPS County Code'] = inc_by_county['GeoFIPS'].str[2:5]

inc = pd.DataFrame()
for year in range(1969,2021):
    inc_oneyear = inc_by_county[['FIPS State Code','FIPS County Code',str(year)]].copy()
    inc_oneyear = inc_oneyear.rename(columns={str(year):'inc'})
    inc_oneyear['year'] = year
    inc = pd.concat([inc,inc_oneyear])

# Note that CBSA should not be used below: It is not a complete list of all counties in the US

# Complete list of counties, including those not part of CSA 
all_counties = pd.read_csv("../RawData/MSA/fips-by-state.csv",sep=',',encoding="ISO-8859-1",low_memory=False)
all_counties = all_counties.rename(columns={'name':'County','state':'State'})
all_counties['County'] = all_counties['County'].str.upper()
all_counties['County'] = all_counties['County'].str.replace(' COUNTY','')
all_counties['County'] = all_counties['County'].str.replace(' AND ',' & ')
all_counties['County'] = all_counties['County'].str.replace('.','',regex=False)

all_counties['fips'] = all_counties['fips'].astype(str)
all_counties.loc[all_counties['fips'].str.len()==4,'fips'] = '0'+all_counties['fips'][all_counties['fips'].str.len()==4]
all_counties['FIPS State Code'] = all_counties['fips'].str.slice(0,2)
all_counties['FIPS County Code'] = all_counties['fips'].str.slice(2,5)

all_counties_Exploded = pd.DataFrame()
for year in range(1969,2021):
    all_counties['year'] = year
    all_counties_Exploded = pd.concat([all_counties_Exploded,all_counties])
all_counties_Exploded['year'] = all_counties_Exploded['year'].astype(str)

# Merge with county data
black_pop = all_counties_Exploded.merge(black_pop,on=['FIPS State Code','FIPS County Code','year'])
black_pop['year'] = black_pop['year'].astype(int)
black_pop = black_pop.merge(inc,on=['FIPS State Code','FIPS County Code','year'])
black_pop.loc[black_pop['inc']=='(NA)','inc'] = None
black_pop['inc'] = black_pop['inc'].astype(float)

# Export data
black_pop['year'] = black_pop['year'].astype(int)
black_pop_2021 = black_pop[black_pop['year']==2020].copy()
black_pop_2021['year'] = 2021
black_pop_2022 = black_pop[black_pop['year']==2020].copy()
black_pop_2022['year'] = 2022
black_pop_2023 = black_pop[black_pop['year']==2020].copy()
black_pop_2023['year'] = 2023
black_pop = pd.concat([black_pop,black_pop_2021,black_pop_2022,black_pop_2023])
black_pop.to_csv("../RawData/MSA/POP/black_pop.csv")

# 4. Geographics

- Get a list of MSAs that has overlap in terms of being in the same state. Use these to rule out unreasonable control groups that might also get affected by the treatment. Note that just by matching using the population and average income will result in some treated-control pairs that are very geographically approximate.

In [ ]:
# "CSA" is for metropolitan and "CBSAData" includes also those micropolitan
CBSAData = pd.read_excel("../RawData/MSA/CBSA.xlsx",skiprows=[0,1])
CBSAData = CBSAData[~pd.isnull(CBSAData['County/County Equivalent'])]

# Add state abbreviations
%run -i SCRIPT_us_states.py
us_state_to_abbrev = pd.DataFrame.from_dict(us_state_to_abbrev,orient='index').reset_index()
us_state_to_abbrev.columns = ['State Name','State']
CBSAData = CBSAData.rename(columns={'County/County Equivalent':'County'})
CBSAData = CBSAData.merge(us_state_to_abbrev,on='State Name',how='outer',indicator=True)
CBSAData = CBSAData[CBSAData['_merge']=='both'].drop(columns=['_merge'])
# Merge is perfect
CBSAData['County'] = CBSAData['County'].str.upper().str.replace(' COUNTY','')


#-----------#
# CSA pairs #
#-----------#

Same_State_CSA_pairs = []
CSAs = list(CBSAData['CSA Code'].unique())
CSAs = [item for item in CSAs if item!=None and str(item)!='nan']
for CSA in CSAs:
    # A list of CSAs in the same state
    States = list(CBSAData[CBSAData['CSA Code']==CSA]['FIPS State Code'].unique())
    for State in States:
        CSAs_same_state = list(CBSAData[CBSAData['FIPS State Code']==State]['CSA Code'].unique())
        CSAs_same_state = [item for item in CSAs_same_state if item!=None and str(item)!='nan' and item!=CSA]
        for item in CSAs_same_state:
            Same_State_CSA_pairs = Same_State_CSA_pairs+[{'CSA_1':CSA,'CSA_2':item}]
            Same_State_CSA_pairs = Same_State_CSA_pairs+[{'CSA_1':item,'CSA_2':CSA}]

# Pairs of CSAs in the same state
Title_Code = CBSAData[['CSA Code','CSA Title']].drop_duplicates()
Title_Code = Title_Code[~pd.isnull(Title_Code['CSA Code'])]
Same_State_CSA_pairs = pd.DataFrame(Same_State_CSA_pairs)
Same_State_CSA_pairs = Same_State_CSA_pairs.merge(Title_Code.rename(columns={'CSA Code':'CSA_1','CSA Title':'Title_1'}),on='CSA_1')
Same_State_CSA_pairs = Same_State_CSA_pairs.merge(Title_Code.rename(columns={'CSA Code':'CSA_2','CSA Title':'Title_2'}),on='CSA_2')

Same_State_CSA_pairs.to_csv("../RawData/MSA/CAINC1/Same_State_CSA_pairs.csv")

#------------#
# CBSA pairs #
#------------#

Same_State_CBSA_pairs = []
CBSAs = list(CBSAData['CBSA Code'].unique())
CBSAs = [item for item in CBSAs if item!=None and str(item)!='nan']
for CBSA in CBSAs:
    # A list of CBSAs in the same state
    States = list(CBSAData[CBSAData['CBSA Code']==CBSA]['FIPS State Code'].unique())
    for State in States:
        CBSAs_same_state = list(CBSAData[CBSAData['FIPS State Code']==State]['CBSA Code'].unique())
        CBSAs_same_state = [item for item in CBSAs_same_state if item!=None and str(item)!='nan' and item!=CBSA]
        for item in CBSAs_same_state:
            Same_State_CBSA_pairs = Same_State_CBSA_pairs+[{'CBSA_1':CBSA,'CBSA_2':item}]
            Same_State_CBSA_pairs = Same_State_CBSA_pairs+[{'CBSA_1':item,'CBSA_2':CBSA}]

# Pairs of CBSAs in the same state
Title_Code = CBSAData[['CBSA Code','CBSA Title']].drop_duplicates()
Title_Code = Title_Code[~pd.isnull(Title_Code['CBSA Code'])]
Same_State_CBSA_pairs = pd.DataFrame(Same_State_CBSA_pairs)
Same_State_CBSA_pairs = Same_State_CBSA_pairs.merge(Title_Code.rename(columns={'CBSA Code':'CBSA_1','CBSA Title':'Title_1'}),on='CBSA_1')
Same_State_CBSA_pairs = Same_State_CBSA_pairs.merge(Title_Code.rename(columns={'CBSA Code':'CBSA_2','CBSA Title':'Title_2'}),on='CBSA_2')

Same_State_CBSA_pairs.to_csv("../RawData/MSA/CAINC1/Same_State_CBSA_pairs.csv")


# 5. Texas & California Data

In [49]:
# For Texas, fee data is available for 2022 only
Local_Issuance_Fee = pd.read_csv('../RawData/Texas/Local_Issuance_Fee.csv')
Local_Issuance = pd.read_csv('../RawData/Texas/Local_Issuance.csv')
Local_Issuance_Fee.value_counts('FiscalYearIssuance')

Estimate a statistical model of issuance fees (advisor fee, insurance premium, ratings fee) as a ratio of amount:
$$\text{Cost}=
\gamma_1\frac{\text{County Income}}{\text{National Average Income}}+\gamma_2\frac{\text{County Population}}{\text{National Average County Population}}+
\delta_{\text{maturity bracket}}+\delta_{\text{amount bracket}}+
\theta_{\text{method of sales}}+\theta_{\text{tax status}}+\theta_{\text{source of repayment}}
$$

In [392]:
##############################
# Import california fee data #
##############################

CDA_All_Data = pd.read_csv('../RawData/California/CDA_All_Data.csv')

CDA_All_Data['County'] = CDA_All_Data['Issuer County'].str.upper()
CDA_All_Data['State'] = 'CA'

CDA_All_Data['AdvisorRatio'] = CDA_All_Data['Financial Advisor Fee']/CDA_All_Data['Principal Amount']
CDA_All_Data['CRRatio'] = CDA_All_Data['Rating Agency Fee']/CDA_All_Data['Principal Amount']
CDA_All_Data['InsureRatio'] = CDA_All_Data['Credit Enhancement Fee']/CDA_All_Data['Principal Amount']

# Winsorize data
upper_limit = np.percentile(CDA_All_Data['AdvisorRatio'][np.logical_not(np.isnan(CDA_All_Data['AdvisorRatio']))],99)
lower_limit = np.percentile(CDA_All_Data['AdvisorRatio'][np.logical_not(np.isnan(CDA_All_Data['AdvisorRatio']))],1)
CDA_All_Data.loc[(CDA_All_Data['AdvisorRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data['AdvisorRatio']))),'AdvisorRatio'] = \
    upper_limit
CDA_All_Data.loc[(CDA_All_Data['AdvisorRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data['AdvisorRatio']))),'AdvisorRatio'] = \
    lower_limit
upper_limit = np.percentile(CDA_All_Data['CRRatio'][np.logical_not(np.isnan(CDA_All_Data['CRRatio']))],99)
lower_limit = np.percentile(CDA_All_Data['CRRatio'][np.logical_not(np.isnan(CDA_All_Data['CRRatio']))],1)
CDA_All_Data.loc[(CDA_All_Data['CRRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data['CRRatio']))),'CRRatio'] = \
    upper_limit
CDA_All_Data.loc[(CDA_All_Data['CRRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data['CRRatio']))),'CRRatio'] = \
    lower_limit
upper_limit = np.percentile(CDA_All_Data['InsureRatio'][np.logical_not(np.isnan(CDA_All_Data['InsureRatio']))],99)
lower_limit = np.percentile(CDA_All_Data['InsureRatio'][np.logical_not(np.isnan(CDA_All_Data['InsureRatio']))],1)
CDA_All_Data.loc[(CDA_All_Data['InsureRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data['InsureRatio']))),'InsureRatio'] = \
    upper_limit
CDA_All_Data.loc[(CDA_All_Data['InsureRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data['InsureRatio']))),'InsureRatio'] = \
    lower_limit

CDA_All_Data['Sale Date'] = pd.to_datetime(CDA_All_Data['Sale Date'])
CDA_All_Data['year'] = CDA_All_Data['Sale Date'].dt.year
CDA_All_Data['Final Maturity Date'] = pd.to_datetime(CDA_All_Data['Final Maturity Date'])
CDA_All_Data['year_maturity'] = CDA_All_Data['Final Maturity Date'].dt.year
CDA_All_Data['maturity_in_years'] = CDA_All_Data['year_maturity']-CDA_All_Data['year']


######################################
# Merge in county level demographics #
######################################

black_pop = pd.read_csv("../RawData/MSA/POP/black_pop.csv")

# Get national average county-level income and 
black_pop_yearlyavg = black_pop.groupby('year').agg({'pop':'mean','inc':'mean'})
black_pop_yearlyavg = black_pop_yearlyavg.rename(columns={'pop':'pop_yearlyavg','inc':'inc_yearlyavg'})
black_pop = black_pop.merge(black_pop_yearlyavg,on=['year'])
black_pop['pop_to_avg'] = black_pop['pop']/black_pop['pop_yearlyavg']
black_pop['inc_to_avg'] = black_pop['inc']/black_pop['inc_yearlyavg']

CDA_All_Data = CDA_All_Data.merge(black_pop[['State','County','year','pop_to_avg','inc_to_avg']],on=['State','County','year'])

CDA_All_Data = CDA_All_Data[[
    'Issuer','Issuer Group','Issuer Type','County','State','year',
    'Principal Amount','maturity_in_years','Debt Type','Federally Taxable','Sale Type (Comp/Neg)',
    'AdvisorRatio','CRRatio','InsureRatio',
    'pop_to_avg','inc_to_avg']]



################################################
# Recreate variables to be consistent with GPF #
################################################

CDA_All_Data['GPF_security_type'] = None
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Tax and Revenue Anticipation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='General Obligation Bond','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Special Assessment Bond','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Certificate of Participation/Leases','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Limited Tax Obligation Bond (Special Tax Bonds)','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Conduit Revenue Bond','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Public Enterprise Revenue Bond','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Tax Allocation Bond','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Revenue Bond','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Public Lease Revenue Bond','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Marks-Roos Loan','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Commercial Paper','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Conduit Revenue Note or Loan (Private Obligor)','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Bond Anticipation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Capital Lease','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Pension Obligation Bonds','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Other Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Sales Tax Revenue Bond','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Tax Allocation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Revenue Anticipation Note','GPF_security_type'] = 'RV'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Other Bond','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='State Agency Loan','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Loan from bank/other institution','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Tax Anticipation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Grant Anticipation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Other Debt','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Promissory Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='General Obligation Note','GPF_security_type'] = 'GO'
CDA_All_Data.loc[CDA_All_Data['Debt Type']=='Revenue Anticipation Warrant','GPF_security_type'] = 'RV'

CDA_All_Data['GPF_taxable_code'] = None
CDA_All_Data.loc[CDA_All_Data['Federally Taxable']=='Federal Tax Flag: E','GPF_taxable_code'] = 'E'
CDA_All_Data.loc[CDA_All_Data['Federally Taxable']=='Federal Tax Flag: T','GPF_taxable_code'] = 'T'
CDA_All_Data.loc[CDA_All_Data['Federally Taxable']=='Subject to Alternative Minimum Tax','GPF_taxable_code'] = 'A'
CDA_All_Data.loc[CDA_All_Data['Federally Taxable']=='Federal Tax Flag: ET','GPF_taxable_code'] = 'E'
CDA_All_Data.loc[CDA_All_Data['Federally Taxable']=='Federal Tax Flag: TE','GPF_taxable_code'] = 'T'

CDA_All_Data['GPF_Bid'] = None
CDA_All_Data.loc[CDA_All_Data['Sale Type (Comp/Neg)']=='Neg','GPF_Bid'] = 'N'
CDA_All_Data.loc[CDA_All_Data['Sale Type (Comp/Neg)']=='Comp','GPF_Bid'] = 'C'

# Adjust inflation
FPCPITOTLZGUSA = pd.read_csv("../RawData/StLouisFed/FPCPITOTLZGUSA.csv")
FPCPITOTLZGUSA['year'] = FPCPITOTLZGUSA['DATE'].str[:4].astype(int)
FPCPITOTLZGUSA = FPCPITOTLZGUSA.sort_values('year',ascending=False).reset_index(drop=True)
scaler = 1
FPCPITOTLZGUSA['scaler'] = None
for idx,row in FPCPITOTLZGUSA.iterrows():
    if idx==0:
        FPCPITOTLZGUSA.at[idx,'scaler'] = 1
    else:
        scaler = scaler*(FPCPITOTLZGUSA.at[idx-1,'FPCPITOTLZGUSA']/100+1)
        FPCPITOTLZGUSA.at[idx,'scaler'] = scaler
FPCPITOTLZGUSA = FPCPITOTLZGUSA[['scaler','year']]

CDA_All_Data = CDA_All_Data.merge(FPCPITOTLZGUSA,on=['year'])
CDA_All_Data['GPF_amount_inf_adjusted'] = CDA_All_Data['Principal Amount']*CDA_All_Data['scaler']

CDA_All_Data['GPF_amount_bracket'] = None
CDA_All_Data.loc[CDA_All_Data['GPF_amount_inf_adjusted']<=1*1000000,
    'GPF_amount_bracket'] = 'Less than 1M'
CDA_All_Data.loc[(CDA_All_Data['GPF_amount_inf_adjusted']>1*1000000)&(CDA_All_Data['GPF_amount_inf_adjusted']<=5*1000000),
    'GPF_amount_bracket'] = '1M to 5M'
CDA_All_Data.loc[(CDA_All_Data['GPF_amount_inf_adjusted']>5*1000000)&(CDA_All_Data['GPF_amount_inf_adjusted']<=10*1000000),
    'GPF_amount_bracket'] = '5M to 10M'
CDA_All_Data.loc[(CDA_All_Data['GPF_amount_inf_adjusted']>10*1000000)&(CDA_All_Data['GPF_amount_inf_adjusted']<=50*1000000),
    'GPF_amount_bracket'] = '10M to 50M'
CDA_All_Data.loc[(CDA_All_Data['GPF_amount_inf_adjusted']>50*1000000)&(CDA_All_Data['GPF_amount_inf_adjusted']<=100*1000000),
    'GPF_amount_bracket'] = '50M to 100M'
CDA_All_Data.loc[CDA_All_Data['GPF_amount_inf_adjusted']>100*1000000,
    'GPF_amount_bracket'] = 'Greater than 100M'

CDA_All_Data['GPF_maturity_bracket'] = None
CDA_All_Data.loc[CDA_All_Data['maturity_in_years']<=2,'GPF_maturity_bracket'] = 'Less then 2y'
CDA_All_Data.loc[(CDA_All_Data['maturity_in_years']>2)&(CDA_All_Data['maturity_in_years']<=5),'GPF_maturity_bracket'] = '2y to 5y'
CDA_All_Data.loc[(CDA_All_Data['maturity_in_years']>5)&(CDA_All_Data['maturity_in_years']<=10),'GPF_maturity_bracket'] = '5y to 10y'
CDA_All_Data.loc[(CDA_All_Data['maturity_in_years']>10)&(CDA_All_Data['maturity_in_years']<=20),'GPF_maturity_bracket'] = '10y to 20y'
CDA_All_Data.loc[(CDA_All_Data['maturity_in_years']>20)&(CDA_All_Data['maturity_in_years']<=30),'GPF_maturity_bracket'] = '20y to 30y'
CDA_All_Data.loc[(CDA_All_Data['maturity_in_years']>30)&(CDA_All_Data['maturity_in_years']<=40),'GPF_maturity_bracket'] = '30y to 40y'
CDA_All_Data.loc[CDA_All_Data['maturity_in_years']>40,'GPF_maturity_bracket'] = 'Greater than 40y'

CDA_All_Data.to_csv('../RawData/California/SumStats.csv')

In [389]:
##############
# Import GPF #
##############

GPF = pd.read_csv("../RawData/SDC/GPF.csv",low_memory=False)

GPF['is_security_type_GO'] = GPF['security_type']=='GO'
GPF['is_security_type_RV'] = GPF['security_type']=='RV'
GPF['is_Bid_C'] = GPF['Bid']=='C'
GPF['is_Bid_N'] = GPF['Bid']=='N'
GPF['is_taxable_code_A'] = GPF['taxable_code']=='A'
GPF['is_taxable_code_E'] = GPF['taxable_code']=='E'
GPF['is_taxable_code_T'] = GPF['taxable_code']=='T'

GPF = GPF.merge(FPCPITOTLZGUSA.rename(columns={'year':'sale_year'}),on=['sale_year'],how='outer',indicator=True)
GPF = GPF[GPF['_merge']!='right_only']
GPF = GPF.drop(columns=['_merge'])
GPF['amount_inf_adjusted'] = GPF['amount']*GPF['scaler']*1000000

GPF['is_amount_Less_than_1M'] = GPF['amount_inf_adjusted']<1*1000000
GPF['is_amount_1M_to_5M'] = (GPF['amount_inf_adjusted']>1*1000000)&(GPF['amount_inf_adjusted']<=5*1000000)
GPF['is_amount_5M_to_10M'] = (GPF['amount_inf_adjusted']>5*1000000)&(GPF['amount_inf_adjusted']<=10*1000000)
GPF['is_amount_10M_to_50M'] = (GPF['amount_inf_adjusted']>10*1000000)&(GPF['amount_inf_adjusted']<=50*1000000)
GPF['is_amount_50M_to_100M'] = (GPF['amount_inf_adjusted']>50*1000000)&(GPF['amount_inf_adjusted']<=100*1000000)
GPF['is_amount_Greater_than_100M'] = GPF['amount_inf_adjusted']>100*1000000

GPF['maturity_in_years'] = np.round(GPF['avg_maturity']/365)

GPF['is_maturity_Less_than_2y'] = GPF['maturity_in_years']<2
GPF['is_maturity_2y_to_5y'] = (GPF['maturity_in_years']>2)&(GPF['maturity_in_years']<=5)
GPF['is_maturity_5y_to_10y'] = (GPF['maturity_in_years']>5)&(GPF['maturity_in_years']<=10)
GPF['is_maturity_10y_to_20y'] = (GPF['maturity_in_years']>10)&(GPF['maturity_in_years']<=20)
GPF['is_maturity_20y_to_30y'] = (GPF['maturity_in_years']>20)&(GPF['maturity_in_years']<=30)
GPF['is_maturity_30y_to_40y'] = (GPF['maturity_in_years']>30)&(GPF['maturity_in_years']<=40)
GPF['is_maturity_Greater_than_40y'] = GPF['maturity_in_years']>40

GPF = GPF.merge(black_pop[['State','County','year','pop_to_avg','inc_to_avg']]\
    .rename(columns={'year':'sale_year'}),on=['State','County','sale_year'],how='outer',indicator=True)
GPF = GPF[GPF['_merge']!='right_only']
GPF = GPF.drop(columns=['_merge'])


In [391]:
# Predict would-be cost of advisors, credit ratings, and insurance

################
# Advisors fee #
################

# Run regression in the California sample
# To make sure that order of first category is fixed
CDA_All_Data = CDA_All_Data.sort_values(['GPF_maturity_bracket','GPF_amount_bracket','GPF_Bid','GPF_security_type','GPF_taxable_code'])
maturity_bracket = pd.get_dummies(CDA_All_Data['GPF_maturity_bracket'], drop_first=True)
amount_bracket = pd.get_dummies(CDA_All_Data['GPF_amount_bracket'], drop_first=True)
Bid = pd.get_dummies(CDA_All_Data['GPF_Bid'], drop_first=True)
security_type = pd.get_dummies(CDA_All_Data['GPF_security_type'], drop_first=True)
taxable_code = pd.get_dummies(CDA_All_Data['GPF_taxable_code'], drop_first=True)

CDA_All_Data_RegData = pd.concat([CDA_All_Data,maturity_bracket,amount_bracket,Bid,security_type,taxable_code],axis=1)
CDA_All_Data_RegData = CDA_All_Data_RegData.dropna(subset=['AdvisorRatio'])
CDA_All_Data_RegData['AdvisorRatio'] = CDA_All_Data_RegData['AdvisorRatio']*10000

upper_limit = np.percentile(CDA_All_Data_RegData['AdvisorRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['AdvisorRatio']))],99)
lower_limit = np.percentile(CDA_All_Data_RegData['AdvisorRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['AdvisorRatio']))],1)
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['AdvisorRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['AdvisorRatio']))),'AdvisorRatio'] = \
    upper_limit
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['AdvisorRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['AdvisorRatio']))),'AdvisorRatio'] = \
    lower_limit

X = CDA_All_Data_RegData[['pop_to_avg','inc_to_avg']
    +list(maturity_bracket.columns)+list(amount_bracket.columns)+list(Bid.columns)+list(security_type.columns)+list(taxable_code.columns)]
y = CDA_All_Data_RegData['AdvisorRatio']

CDA_All_Data_RegData.to_csv("../RawData/California/CDA_All_Data_RegData_AdvisorRatio.csv")

model = sm.OLS(y, sm.add_constant(X))
result = model.fit()

# Predict would-be costs
GPF['AdvisorRatio_hat'] = result.params['const']+\
    result.params['pop_to_avg']*GPF['pop_to_avg']+\
    result.params['inc_to_avg']*GPF['inc_to_avg']+\
    result.params['20y to 30y']*GPF['is_maturity_20y_to_30y']+\
    result.params['2y to 5y']*GPF['is_maturity_2y_to_5y']+\
    result.params['30y to 40y']*GPF['is_maturity_30y_to_40y']+\
    result.params['5y to 10y']*GPF['is_maturity_5y_to_10y']+\
    result.params['Greater than 40y']*GPF['is_maturity_Greater_than_40y']+\
    result.params['Less then 2y']*GPF['is_maturity_Less_than_2y']+\
    result.params['1M to 5M']*GPF['is_amount_1M_to_5M']+\
    result.params['50M to 100M']*GPF['is_amount_50M_to_100M']+\
    result.params['5M to 10M']*GPF['is_amount_5M_to_10M']+\
    result.params['Greater than 100M']*GPF['is_amount_Greater_than_100M']+\
    result.params['Less than 1M']*GPF['is_amount_Less_than_1M']+\
    result.params['N']*GPF['is_Bid_N']+\
    result.params['RV']*GPF['is_security_type_RV']+\
    result.params['E']*GPF['is_taxable_code_E']+\
    result.params['T']*GPF['is_taxable_code_T']

GPF.loc[GPF['AdvisorRatio_hat']<0,'AdvisorRatio_hat'] = 0
GPF['AdvisorRatio_hat'] = GPF['AdvisorRatio_hat'].astype(float)

upper_limit = np.percentile(GPF['AdvisorRatio_hat'][np.logical_not(np.isnan(GPF['AdvisorRatio_hat']))],99)
lower_limit = np.percentile(GPF['AdvisorRatio_hat'][np.logical_not(np.isnan(GPF['AdvisorRatio_hat']))],1)
GPF.loc[(GPF['AdvisorRatio_hat']>upper_limit)&(np.logical_not(np.isnan(GPF['AdvisorRatio_hat']))),'AdvisorRatio_hat'] = \
    upper_limit
GPF.loc[(GPF['AdvisorRatio_hat']<lower_limit)&(np.logical_not(np.isnan(GPF['AdvisorRatio_hat']))),'AdvisorRatio_hat'] = \
    lower_limit



#####################
# Credit rating fee #
#####################

# Run regression in the California sample
# To make sure that order of first category is fixed
CDA_All_Data = CDA_All_Data.sort_values(['GPF_maturity_bracket','GPF_amount_bracket','GPF_Bid','GPF_security_type','GPF_taxable_code'])
maturity_bracket = pd.get_dummies(CDA_All_Data['GPF_maturity_bracket'], drop_first=True)
amount_bracket = pd.get_dummies(CDA_All_Data['GPF_amount_bracket'], drop_first=True)
Bid = pd.get_dummies(CDA_All_Data['GPF_Bid'], drop_first=True)
security_type = pd.get_dummies(CDA_All_Data['GPF_security_type'], drop_first=True)
taxable_code = pd.get_dummies(CDA_All_Data['GPF_taxable_code'], drop_first=True)

CDA_All_Data_RegData = pd.concat([CDA_All_Data,maturity_bracket,amount_bracket,Bid,security_type,taxable_code],axis=1)
CDA_All_Data_RegData = CDA_All_Data_RegData.dropna(subset=['CRRatio'])
CDA_All_Data_RegData['CRRatio'] = CDA_All_Data_RegData['CRRatio']*10000

upper_limit = np.percentile(CDA_All_Data_RegData['CRRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['CRRatio']))],99)
lower_limit = np.percentile(CDA_All_Data_RegData['CRRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['CRRatio']))],1)
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['CRRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['CRRatio']))),'CRRatio'] = \
    upper_limit
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['CRRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['CRRatio']))),'CRRatio'] = \
    lower_limit

X = CDA_All_Data_RegData[['pop_to_avg','inc_to_avg']
    +list(maturity_bracket.columns)+list(amount_bracket.columns)+list(Bid.columns)+list(security_type.columns)+list(taxable_code.columns)]
y = CDA_All_Data_RegData['CRRatio']

CDA_All_Data_RegData.to_csv("../RawData/California/CDA_All_Data_RegData_CRRatio.csv")

model = sm.OLS(y, sm.add_constant(X))
result = model.fit()

# Predict would-be costs
GPF['CRRatio_hat'] = result.params['const']+\
    result.params['pop_to_avg']*GPF['pop_to_avg']+\
    result.params['inc_to_avg']*GPF['inc_to_avg']+\
    result.params['20y to 30y']*GPF['is_maturity_20y_to_30y']+\
    result.params['2y to 5y']*GPF['is_maturity_2y_to_5y']+\
    result.params['30y to 40y']*GPF['is_maturity_30y_to_40y']+\
    result.params['5y to 10y']*GPF['is_maturity_5y_to_10y']+\
    result.params['Greater than 40y']*GPF['is_maturity_Greater_than_40y']+\
    result.params['Less then 2y']*GPF['is_maturity_Less_than_2y']+\
    result.params['1M to 5M']*GPF['is_amount_1M_to_5M']+\
    result.params['50M to 100M']*GPF['is_amount_50M_to_100M']+\
    result.params['5M to 10M']*GPF['is_amount_5M_to_10M']+\
    result.params['Greater than 100M']*GPF['is_amount_Greater_than_100M']+\
    result.params['Less than 1M']*GPF['is_amount_Less_than_1M']+\
    result.params['N']*GPF['is_Bid_N']+\
    result.params['RV']*GPF['is_security_type_RV']+\
    result.params['E']*GPF['is_taxable_code_E']+\
    result.params['T']*GPF['is_taxable_code_T']

GPF.loc[GPF['CRRatio_hat']<0,'CRRatio_hat'] = 0
GPF['CRRatio_hat'] = GPF['CRRatio_hat'].astype(float)

upper_limit = np.percentile(GPF['CRRatio_hat'][np.logical_not(np.isnan(GPF['CRRatio_hat']))],99)
lower_limit = np.percentile(GPF['CRRatio_hat'][np.logical_not(np.isnan(GPF['CRRatio_hat']))],1)
GPF.loc[(GPF['CRRatio_hat']>upper_limit)&(np.logical_not(np.isnan(GPF['CRRatio_hat']))),'CRRatio_hat'] = \
    upper_limit
GPF.loc[(GPF['CRRatio_hat']<lower_limit)&(np.logical_not(np.isnan(GPF['CRRatio_hat']))),'CRRatio_hat'] = \
    lower_limit




#################
# Insurance fee #
#################

# Run regression in the California sample
# To make sure that order of first category is fixed
CDA_All_Data = CDA_All_Data.sort_values(['GPF_maturity_bracket','GPF_amount_bracket','GPF_Bid','GPF_security_type','GPF_taxable_code'])
maturity_bracket = pd.get_dummies(CDA_All_Data['GPF_maturity_bracket'], drop_first=True)
amount_bracket = pd.get_dummies(CDA_All_Data['GPF_amount_bracket'], drop_first=True)
Bid = pd.get_dummies(CDA_All_Data['GPF_Bid'], drop_first=True)
security_type = pd.get_dummies(CDA_All_Data['GPF_security_type'], drop_first=True)
taxable_code = pd.get_dummies(CDA_All_Data['GPF_taxable_code'], drop_first=True)

CDA_All_Data_RegData = pd.concat([CDA_All_Data,maturity_bracket,amount_bracket,Bid,security_type,taxable_code],axis=1)
CDA_All_Data_RegData = CDA_All_Data_RegData.dropna(subset=['InsureRatio'])
CDA_All_Data_RegData['InsureRatio'] = CDA_All_Data_RegData['InsureRatio']*10000

upper_limit = np.percentile(CDA_All_Data_RegData['InsureRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['InsureRatio']))],99)
lower_limit = np.percentile(CDA_All_Data_RegData['InsureRatio'][np.logical_not(np.isnan(CDA_All_Data_RegData['InsureRatio']))],1)
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['InsureRatio']>upper_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['InsureRatio']))),'InsureRatio'] = \
    upper_limit
CDA_All_Data_RegData.loc[(CDA_All_Data_RegData['InsureRatio']<lower_limit)&(np.logical_not(np.isnan(CDA_All_Data_RegData['InsureRatio']))),'InsureRatio'] = \
    lower_limit

X = CDA_All_Data_RegData[['pop_to_avg','inc_to_avg']
    +list(maturity_bracket.columns)+list(amount_bracket.columns)+list(Bid.columns)+list(security_type.columns)+list(taxable_code.columns)]
y = CDA_All_Data_RegData['InsureRatio']

CDA_All_Data_RegData.to_csv("../RawData/California/CDA_All_Data_RegData_InsureRatio.csv")

model = sm.OLS(y, sm.add_constant(X))
result = model.fit()

# Predict would-be costs
GPF['InsureRatio_hat'] = result.params['const']+\
    result.params['pop_to_avg']*GPF['pop_to_avg']+\
    result.params['inc_to_avg']*GPF['inc_to_avg']+\
    result.params['20y to 30y']*GPF['is_maturity_20y_to_30y']+\
    result.params['2y to 5y']*GPF['is_maturity_2y_to_5y']+\
    result.params['30y to 40y']*GPF['is_maturity_30y_to_40y']+\
    result.params['5y to 10y']*GPF['is_maturity_5y_to_10y']+\
    result.params['Greater than 40y']*GPF['is_maturity_Greater_than_40y']+\
    result.params['Less then 2y']*GPF['is_maturity_Less_than_2y']+\
    result.params['1M to 5M']*GPF['is_amount_1M_to_5M']+\
    result.params['50M to 100M']*GPF['is_amount_50M_to_100M']+\
    result.params['5M to 10M']*GPF['is_amount_5M_to_10M']+\
    result.params['Greater than 100M']*GPF['is_amount_Greater_than_100M']+\
    result.params['Less than 1M']*GPF['is_amount_Less_than_1M']+\
    result.params['N']*GPF['is_Bid_N']+\
    result.params['RV']*GPF['is_security_type_RV']+\
    result.params['E']*GPF['is_taxable_code_E']+\
    result.params['T']*GPF['is_taxable_code_T']

GPF.loc[GPF['InsureRatio_hat']<0,'InsureRatio_hat'] = 0
GPF['InsureRatio_hat'] = GPF['InsureRatio_hat'].astype(float)

upper_limit = np.percentile(GPF['InsureRatio_hat'][np.logical_not(np.isnan(GPF['InsureRatio_hat']))],99)
lower_limit = np.percentile(GPF['InsureRatio_hat'][np.logical_not(np.isnan(GPF['InsureRatio_hat']))],1)
GPF.loc[(GPF['InsureRatio_hat']>upper_limit)&(np.logical_not(np.isnan(GPF['InsureRatio_hat']))),'InsureRatio_hat'] = \
    upper_limit
GPF.loc[(GPF['InsureRatio_hat']<lower_limit)&(np.logical_not(np.isnan(GPF['InsureRatio_hat']))),'InsureRatio_hat'] = \
    lower_limit


In [ ]:
#--------------------------------------------------#
# Export a version of GPF with imputed other costs #
#--------------------------------------------------#

GPF = GPF.drop(columns=[
    'is_security_type_GO', 'is_security_type_RV', 'is_Bid_C', 'is_Bid_N',
    'is_taxable_code_A', 'is_taxable_code_E', 'is_taxable_code_T', 'scaler',
    'amount_inf_adjusted', 'is_amount_Less_than_1M', 'is_amount_1M_to_5M',
    'is_amount_5M_to_10M', 'is_amount_10M_to_50M', 'is_amount_50M_to_100M',
    'is_amount_Greater_than_100M', 'maturity_in_years',
    'is_maturity_Less_than_2y', 'is_maturity_2y_to_5y',
    'is_maturity_5y_to_10y', 'is_maturity_10y_to_20y',
    'is_maturity_20y_to_30y', 'is_maturity_30y_to_40y',
    'is_maturity_Greater_than_40y', 'pop_to_avg', 'inc_to_avg',
    ])
GPF.to_csv("../RawData/SDC/GPF.csv")